Group 09 <br>
BI Assignment 6 Solution <br>
Members: Sana Afreen & Yordanos Girma

# Exercise 1

<div align="center">

# ==========================================
# Exercise 1a: Generator Pipeline
# ==========================================

</div>



Create a Pipeline (Generator), that is given a .bpmn file and generates a descriptive text for the given BPMN-model. The prompt should also contain a BPMN-text pair as an example (if you want to use annotations for the evaluation, you can include them in the generation). (2 Points)

• Input: BPMN-file<br>
• Output: Descriptive Text

In [1]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders.chat_prompt_builder import ChatPromptBuilder
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.dataclasses import ChatMessage
from typing import List, Dict, Any
from haystack import Pipeline
from haystack import Document
from haystack import component
from pathlib import Path
import re

/usr/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
document_store = InMemoryDocumentStore()

# List of file paths
file_paths = [
    "Exces_bagagePair.txt",
    "Passenger Securitypair.txt",  
    "Delayed BaggagePair.txt",
    "Passenger ArrivalPair.txt"   
]


docs = []
for file_path in file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
        docs.append(Document(content=text))


doc_embedder = SentenceTransformersDocumentEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)
doc_embedder.warm_up()
embedded_docs = doc_embedder.run(docs)

document_store.write_documents(embedded_docs["documents"])

retriever = InMemoryEmbeddingRetriever(document_store=document_store)
text_embedder = SentenceTransformersTextEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)

Batches: 100%|██████████| 1/1 [00:00<00:00,  7.41it/s]


In [3]:
print(f"Documents in store: {len(document_store.filter_documents())}")

Documents in store: 4


In [ ]:
PROMPT_TEMPLATE = """

You are a BPMN modeling expert. 
Generate a natural language description of the provided BPMN file by including the BPMN element names 
Do NOT describe the process without naming the BPMN elements by their HUMAN-READABLE NAMES.

CRITICAL RULES:
- ALWAYS use the BPMN element **name attribute**, NOT the XML id.
- NEVER use technical IDs like Task_123, Gateway_ABC, Event_1.
- If an element has no name, describe it generically (e.g., "Unnamed Task", "Unnamed Exclusive Gateway").

For every element, explicitly state its type using these exact terms:
- "Start Event", "End Event"
- "Task" 
- "Exclusive Gateway", "Parallel Gateway"

Use this bpmn text pair description as an Example:
{{ example }}

### Current BPMN to describe:
{{ bpmn_content }}

Descriptive Text:
"""
prompt_builder = PromptBuilder(
    template=PROMPT_TEMPLATE, 
    required_variables=["bpmn_content","example"] 
)

In [5]:
generator = OllamaGenerator(
    model="llama3.2",          
    url="http://localhost:11434",
    timeout=1800,                 
    generation_kwargs={
        "num_ctx": 4096,         
        "temperature": 0.7       
    }
)

In [ ]:
generator_pipeline = Pipeline()
generator_pipeline.add_component("text_embedder", text_embedder)
generator_pipeline.add_component("retriever", retriever)
generator_pipeline.add_component("prompt_builder", prompt_builder)
generator_pipeline.add_component("generator", generator)

generator_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
generator_pipeline.connect("retriever.documents", "prompt_builder.example")
generator_pipeline.connect("prompt_builder.prompt", "generator.prompt")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - generator: OllamaGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (list[float])
  - retriever.documents -> prompt_builder.example (list[Document])
  - prompt_builder.prompt -> generator.prompt (str)

In [ ]:
import os
file_path = "passenger_check_in_machine.bpmn"

if os.path.exists(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        bpmn_xml = f.read()
    
    results = generator_pipeline.run({
        "text_embedder": {"text": bpmn_xml},
        "prompt_builder": {"bpmn_content": bpmn_xml}
    })
    
    descriptive_text = results["generator"]["replies"][0]
    print("--- Generated Description ---\n")
    print(descriptive_text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 34.60it/s]


--- Generated Description ---

Passenger Security Process

The process begins when a passenger shows their boarding pass. Next, the passenger enters the terminal (Task: Passengr enters terminal) and then proceeds to the machine (Task: Passengr goes to mashine). After that, the passenger has to identify (Task: Passengr has to identif) their baggage.

The process then splits into two paths: one for passengers with baggage (Exclusive Gateway: Does the passanger have baggage?) and one without baggage. 

Passengers with baggage proceed to check-in at the counter (Task: Passengr goes to baggage claim counter), after which they receive their boarding pass (Task: Passengr gets Boarding Pass). On the other hand, passengers without baggage go straight to security checks (Task: Passengr goes to security check).

Both paths end with a passenger arriving at the destination airport. The process concludes when all passengers have completed either the check-in and security processes or have proceeded 

<div align="center">

# ==========================================
# Exercise 1b: Extractor Pipeline
# ==========================================

</div>

Create a Pipeline (Extractor), that evaluates a given text using a method from exercise sheet 4 by extracting all tasks (, events and gateways) from the text. (2 Points) 

• Input: Descriptive Text<br>
• Output: List of element names

In [8]:
import json 
from pydantic import BaseModel, ValidationError
from typing import List, Type, Optional

# 1. Define Data Structure (Exercise Sheet 4, Page 4)
class BPMNElements(BaseModel):
    tasks: List[str]
    events: List[str]
    gateways: List[str]

In [9]:
# 2. Custom OutputValidator with Regex Cleaning
@component
class OutputValidator:
    def __init__(self, pydantic_model: Type[BaseModel]):
        self.pydantic_model = pydantic_model

    @component.output_types(
        valid_replies=List[str], 
        invalid_replies=Optional[List[str]], 
        error_message=Optional[str], 
        validated_output=Optional[BPMNElements]
    )
    def run(self, replies: List[str]):
        raw_text = replies[0]
        # Extracts only the JSON portion to avoid conversational filler
        json_match = re.search(r"\{.*\}", raw_text, re.DOTALL)
        
        if not json_match:
            return {
                "invalid_replies": replies,
                "error_message": "No JSON object found. Please return only the JSON."
            }
        
        clean_json = json_match.group(0)

        try:
            output = self.pydantic_model.model_validate_json(clean_json)
            return {"valid_replies": [clean_json], "validated_output": output}
        except (ValidationError, ValueError) as e:
            return {
                "invalid_replies": replies, 
                "error_message": f"Schema mismatch: {str(e)}"
            }

In [ ]:
extractor_validator = OutputValidator(pydantic_model=BPMNElements)

In [ ]:
EXTRACTION_PROMPT = """
Extract all BPMN tasks, events, and gateways from the following text: {{ passage }}

Return ONLY a JSON object with these keys: "tasks", "events", "gateways".
JSON Schema: {{ schema }}

{% if invalid_replies and error_message %}
Your previous attempt failed: {{ invalid_replies[0] }}
Error: {{ error_message }}
Fix the JSON and return only the valid object.
{% endif %}
"""

extractor_prompt_builder = PromptBuilder(
    template=EXTRACTION_PROMPT,
    required_variables=["passage", "schema"]
)

In [ ]:
extractor_generator = OllamaGenerator(
    model="llama3.2",
    url="http://localhost:11434",
    generation_kwargs={
        "format": "json", 
        "temperature": 0
    }
)

In [ ]:
extractor_pipeline = Pipeline(max_runs_per_component=10)
extractor_pipeline.add_component("prompt_builder", extractor_prompt_builder)
extractor_pipeline.add_component("llm", extractor_generator)
extractor_pipeline.add_component("validator", extractor_validator)

extractor_pipeline.connect("prompt_builder", "llm")
extractor_pipeline.connect("llm.replies", "validator.replies")

extractor_pipeline.connect("validator.invalid_replies", "prompt_builder.invalid_replies")
extractor_pipeline.connect("validator.error_message", "prompt_builder.error_message")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: OllamaGenerator
  - validator: OutputValidator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)
  - llm.replies -> validator.replies (List[str])
  - validator.invalid_replies -> prompt_builder.invalid_replies (Optional[List[str]])
  - validator.error_message -> prompt_builder.error_message (Optional[str])

In [ ]:
result = extractor_pipeline.run({
    "prompt_builder": {
        "passage": descriptive_text,
        "schema": json.dumps(BPMNElements.model_json_schema())
    }
})

# 7. Refined result extraction
if result.get("validator") and result["validator"].get("validated_output"):
    extracted_elements = result["validator"]["validated_output"]
    print("--- Extracted Element Names ---")
    print(f"Tasks: {extracted_elements.tasks}")
    print(f"Events: {extracted_elements.events}")
    print(f"Gateways: {extracted_elements.gateways}")
else:
    print("Failed to extract structured data after maximum retries.")

--- Extracted Element Names ---
Tasks: ['Passengr enters terminal', 'Passengr goes to mashine', 'Passengr has to identif', 'Passengr goes to baggage claim counter', 'Passengr gets Boarding Pass']
Events: []
Gateways: ['Does the passanger have baggage?']


<div align="center">

# ==========================================
# Exercise 1c: Evaluation Pipeline
# ==========================================

</div>

Write a function that receives the list of element names from the Extractor-Pipeline and the element names extracted from the BPMN-file used in exercise a) and returns mistakes. (2 Points)

In [ ]:
from typing import Set

# 1. Define the Ground Truth Extractor
@component
class BPMNElementExtractor():
    @component.output_types(extracted_names=Set[str])
    def run(self, path: str):
        # Read the raw XML content from the BPMN file
        bpmn_content = Path(path).read_text(encoding='utf-8')
        
        # Finds names of tasks, events, and gateways
        pattern = re.compile(r'<bpmn:(startEvent|task|exclusiveGateway|parallelGateway|endEvent)\b[^>]*\bname="([^"]+)"')
        matches = pattern.findall(bpmn_content)
        
        # Extract the name attribute (the second group in the regex match)
        bpmn_names = {match[1] for match in matches}
            
        return {"extracted_names": bpmn_names}

In [ ]:
# Extract Ground Truth Names from the File

bpmn_file_extractor = BPMNElementExtractor()


file_path = "passenger_check_in_machine.bpmn"

# Execute the extraction
ground_truth_result = bpmn_file_extractor.run(path=file_path)
ground_truth_names = ground_truth_result["extracted_names"]

print(f"Ground Truth Elements from XML: {ground_truth_names}")

Ground Truth Elements from XML: {'Passenger chooses seat', 'Passenger goes to security check', 'Passenger goes to the mashine', 'Does the passenger have baggage?', 'Passenger gets Boarding Pass', 'Passenger has to identify', 'Passenger enters terminal', 'Passenger goes to baggage claim counter'}


In [ ]:
def compare_elements(extracted_elements, ground_truth_names):
    """
    Compare elements extracted from text with ground truth BPMN XML names.
    Returns missing and hallucinated elements.
    """

    # Combine all extracted elements from LLM
    llm_found_names = set(
        extracted_elements.tasks +
        extracted_elements.events +
        extracted_elements.gateways
    )

    # Identify missing and hallucinated elements
    missing_elements = ground_truth_names - llm_found_names
    hallucinated_elements = llm_found_names - ground_truth_names

    return {
        "missing": list(missing_elements),
        "hallucinated": list(hallucinated_elements)
    }
    


<div align="center">

# ==========================================
# Exercise 1d: Corrector Pipeline
# ==========================================

</div>

Create a Pipeline (Corrector), that is given the mistakes that were found in the text and instructs an LLM to correct them (from exercise sheet 5). (2 Points)

• Input: Mistakes<br>
• Output: Corrected Text

In [ ]:
CORRECTOR_PROMPT = """
You are an expert in business processes. You have previously created a description of a BPMN model, but it contains mistakes.

Initial Description:
{{ initial_description }}

Mistakes identified:
- Missing elements (Add these to the text): {{ missing_elements }}
- Hallucinated elements (Remove these from the text): {{ hallucinated_elements }}

Please provide a corrected version of the process description. Ensure all missing elements are included logically and all hallucinated elements are removed. 

Corrected Description:
"""
# 2. Setup Components for the Corrector Pipeline
corrector_prompt_builder = PromptBuilder(
    template=CORRECTOR_PROMPT,
    required_variables=["initial_description", "missing_elements", "hallucinated_elements"]
)

In [ ]:

corrector_generator = OllamaGenerator(
    model="llama3.2",
    url="http://localhost:11434",
    generation_kwargs={
        "num_ctx": 4096,
        "temperature": 0.2 
    }
)

In [ ]:
# 3. Build the Corrector Pipeline
corrector_pipeline = Pipeline()
corrector_pipeline.add_component("prompt_builder", corrector_prompt_builder)
corrector_pipeline.add_component("llm", corrector_generator)

corrector_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: OllamaGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [ ]:
# We use 'descriptive_text' (from Ex 1a) 
correction_result = corrector_pipeline.run({
    "prompt_builder": {
        "initial_description": descriptive_text,
        "missing_elements": ["missing"],
        "hallucinated_elements": ["hallucinated"]
    }
})
# Display the Final Corrected Text
final_description = correction_result["llm"]["replies"][0]

print("--- Final Corrected Process Description ---")
print(final_description)

--- Final Corrected Process Description ---
Passenger Security Process

The process begins when a passenger shows their boarding pass. Next, the passenger enters the terminal (Task: Passengr enters terminal). The task is triggered by the arrival of the passenger at the airport.

After entering the terminal, the passenger proceeds to the machine (Task: Passengr goes to mashine), where they present their boarding pass for verification. This step is necessary to ensure that the passenger has a valid ticket and can proceed with the security process.

The next task is for the passenger to identify (Task: Passengr has to identif) their baggage, which is stored in a designated area. This task is crucial as it allows the airline staff to prepare for the passenger's arrival at the gate.

The process then splits into two paths: one for passengers with baggage (Exclusive Gateway: Does the passanger have baggage?) and one without baggage.

Passengers with baggage proceed to check-in at the counter

<div align="center">

# ==========================================
# Exercise 2 and 3 
Write Python Code to connect the Pipelines by using the Output of a Pipeline as the Input of the next Pipeline.
Run the code for 3 different BPMN models
# ==========================================

</div>

In [24]:
BPMN_FILES = [
    "passenger_check_in_machine.bpmn",
    "passenger_check_in_counter.bpmn",
    "baggage_departure.bpmn"
]

MAX_ITERATIONS = 5

all_results = {}

for bpmn_file in BPMN_FILES:
    print("\n" + "="*100)
    print(f"RUNNING FULL PIPELINE FOR: {bpmn_file}")
    print("="*100)

   
    with open(bpmn_file, "r", encoding="utf-8") as f:
        bpmn_xml = f.read()

    # Extract Ground Truth
    ground_truth_result = bpmn_file_extractor.run(path=bpmn_file)
    ground_truth_names = ground_truth_result["extracted_names"]

    print("\n--- Ground Truth BPMN Elements ---")
    print(ground_truth_names)

    run_log = {
        "initial_description": None,
        "iterations": [],
        "final_description": None,
        "converged": False
    }


    # Generator → Extractor (Initial Generation)
    gen_result = generator_pipeline.run({
        "text_embedder": {"text": bpmn_xml},
        "prompt_builder": {"bpmn_content": bpmn_xml}
    })

    current_text = gen_result["generator"]["replies"][0]
    run_log["initial_description"] = current_text

    print("\n================ INITIAL GENERATED TEXT ================")
    print(current_text)
    print("========================================================")

    #  Iterative Loop
    for iteration in range(1, MAX_ITERATIONS + 1):
        print(f"\n################ ITERATION {iteration} ################")

        # Extract
        extract_result = extractor_pipeline.run({
            "prompt_builder": {
                "passage": current_text,
                "schema": json.dumps(BPMNElements.model_json_schema())
            }
        })

        if not extract_result.get("validator") or not extract_result["validator"].get("validated_output"):
            print("❌ Extraction failed — stopping.")
            break

        extracted_elements = extract_result["validator"]["validated_output"]

        # Compare (Mistake Detection)
        mistakes_found = compare_elements(
            extracted_elements,
            ground_truth_names
        )

        print("\n--- Mistakes Found ---")
        print("❌ Missing:", mistakes_found["missing"])
        print("⚠️ Hallucinated:", mistakes_found["hallucinated"])

        iteration_log = {
            "iteration": iteration,
            "mistakes": mistakes_found,
            "corrected_text": None
        }

        # Stop if no mistakes-
        if not mistakes_found["missing"] and not mistakes_found["hallucinated"]:
            print("\n✅ No mistakes found — Converged!")
            run_log["converged"] = True
            run_log["final_description"] = current_text
            run_log["iterations"].append(iteration_log)
            break

        # Correct
        correction_result = corrector_pipeline.run({
            "prompt_builder": {
                "initial_description": current_text,
                "missing_elements": mistakes_found["missing"],
                "hallucinated_elements": mistakes_found["hallucinated"]
            }
        })

        corrected_text = correction_result["llm"]["replies"][0]
        iteration_log["corrected_text"] = corrected_text

        print("\n============= GENERATED TEXT AFTER CORRECTOR =============")
        print(corrected_text)
        print("==========================================================")

        run_log["iterations"].append(iteration_log)

        # Feed back into loop
        current_text = corrected_text

        if iteration == MAX_ITERATIONS:
            print("\n⚠️ Max iterations reached.")
            run_log["final_description"] = current_text

    all_results[bpmn_file] = run_log


print("\n" + "="*100)
print("EXERCISE 3 — TESTING RESULTS (REPORT FORMAT)")
print("="*100)

for bpmn_file, log in all_results.items():
    print("\n" + "#"*100)
    print(f"BPMN MODEL: {bpmn_file}")
    print("#"*100)

    # Initially Generated Text
    print("\n--- Initially Generated Text ---")
    print(log["initial_description"])

    # Iterations
    for it in log["iterations"]:
        print(f"\n--- Iteration {it['iteration']} ---")
        print("Mistakes Found:")
        print("  Missing:", it["mistakes"]["missing"])
        print("  Hallucinated:", it["mistakes"]["hallucinated"])

        if it["corrected_text"]:
            print("\nGenerated Text After Corrector:")
            print(it["corrected_text"])

    # Final Status
    print("\n--- Final Status ---")
    print("Converged:", log["converged"])
    print("Final Description:")
    print(log["final_description"])



RUNNING FULL PIPELINE FOR: passenger_check_in_machine.bpmn

--- Ground Truth BPMN Elements ---
{'Passenger chooses seat', 'Passenger goes to security check', 'Passenger goes to the mashine', 'Does the passenger have baggage?', 'Passenger gets Boarding Pass', 'Passenger has to identify', 'Passenger enters terminal', 'Passenger goes to baggage claim counter'}


Batches: 100%|██████████| 1/1 [00:00<00:00, 37.39it/s]



================ INITIAL GENERATED TEXT ================
Passenger Security Process:

The process begins with a **Start Event** named "Arrival at Terminal" and an **Exclusive Gateway** named "Does the passenger have baggage?". When the flight check-in starts, another **Start Event** called "Flight Check-in Begins" occurs. This triggers a sequence of tasks, including:
- An unnamed task where the passenger enters the terminal.
- A task named "Passenger goes to the machine".
- Another task named "Passenger has to identify".

Depending on the baggage situation (Yes/No), the process branches into two paths:

- If the passenger has baggage, a **Parallel Gateway** named "Excess Baggage" takes over. This leads to another exclusive gateway ("Excess Baggage") and subsequently triggers tasks like:
  - An unnamed task where the passenger gets their boarding pass.
  - A task named "Passenger goes to security".
- If there is no excess baggage, a different set of tasks unfolds.

Ultimately, if every

Batches: 100%|██████████| 1/1 [00:00<00:00, 33.02it/s]



================ INITIAL GENERATED TEXT ================
Passenger Security Process

The process begins when a passenger shows their boarding pass. Next, the passenger goes to the counter and has to identify.

The process continues with the passenger choosing a seat. Depending on whether the passenger has luggage or not, they may have to take it. If they do, they get their boarding pass; otherwise, they go through security check. Finally, the process ends when the passenger goes through security check.

Here's how it breaks down:

*   A **Start Event** triggers the process.
*   The process continues with a series of tasks: entering terminal, going to the counter, identifying (for passengers without luggage), and choosing a seat.
*   An **Exclusive Gateway** is used to decide whether the passenger has luggage or not. If they do, they proceed to take their luggage; otherwise, they get their boarding pass.
*   Another **Exclusive Gateway** is used to determine the next step: either takin

Batches: 100%|██████████| 1/1 [00:00<00:00, 31.84it/s]



================ INITIAL GENERATED TEXT ================
This is an XML file that defines a Business Process Model and Notation (BPMN) process diagram. The file contains the definitions for a process with multiple activities, decision points, and edges between them.

Here's a breakdown of the key elements:

* `bpmn:definitions`: This element defines the root of the BPMN process diagram.
* `bpmn:process`: This element represents the overall process, which in this case is unnamed.
* `bpmn:startEvent`: This element marks the beginning of the process. In this case, it's labeled as "Start Event".
* `bpmn:endEvent`: This element marks the end of the process. In this case, it's labeled as "End Event".
* `bpmn:sequenceFlow`: These elements define the edges between activities and decision points.
* `bpmn:activity`: These elements represent individual activities within the process.
* `bpmn:decisionElement`: These elements represent decision points within the process.

Some of the notable activi